In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        """
        Convolutional Neural Network
        
        ネットワーク構成：
            input - CONV - CONV - MaxPool - CONV - CONV - MaxPool - FC - output
            ※MaxPoolの直後にバッチ正規化を実施
 
        引数：
            num_classes: 分類するクラス数（＝出力層のユニット数）
        """
        
        super(CNN, self).__init__() # nn.Moduleのinitを継承
        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.BatchNorm2d(16)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1), # 出力サイズ: チャネル=32, 高さ=26, 幅=26
            nn.BatchNorm2d(32)
        )
        self.full_connection = nn.Sequential(
            nn.Linear(in_features=32*26*26, out_features=512), # in_featuresは直前の出力ユニット数
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=512, out_features=num_classes)
        )
        
        
    # Forward計算の定義
    # 参考：Define by Runの特徴（入力に合わせてForward計算を変更可）
    def forward(self, x):
 
        x = self.block1(x)
        x = self.block2(x)
 
        # 直前のMaxPoolの出力が2次元（×チャネル数）なので，全結合の入力形式に変換
        # 参考：KerasのFlatten()と同じような処理
        x = x.view(x.size(0), 32 * 26 * 26)
 
        y = self.full_connection(x)
        
        return y

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
# modelの定義とパラメータ名とかの関係：確認テスト
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = " + str(device))
print("-------------------------------------------------\n")

print("*****Network1*****")
model1 = CNN(num_classes=10).to(device)
model1.full_connection[3] = nn.Linear(in_features=512, out_features=5)
print(model1) # ネットワークの詳細を確認用に表示
print("*****named_parameters*****")
for name, param in model1.named_parameters():
    print(name)

device = cuda
-------------------------------------------------

*****Network1*****
CNN(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (full_connection): Sequential(
    (0): Linear(in_features=21632, out_features=512, bias=True)
 

In [14]:
# modelの定義とパラメータ名とかの関係：確認テスト
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = " + str(device))
print("-------------------------------------------------\n")

print("*****Network2*****")
model2 = Net().to(device)
model2.fc3 = nn.Linear(in_features=84, out_features=5, bias=True) # 出力層付け替え例
print(model2) # ネットワークの詳細を確認用に表示
print("*****named_parameters*****")
for name, param in model2.named_parameters():
    print(name)

device = cuda
-------------------------------------------------

*****Network2*****
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)
*****named_parameters*****
conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias
